In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt            # library for visualization
from sklearn.model_selection import train_test_split
import functions as fun

[nltk_data] Downloading package punkt to /Users/marta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/marta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("data/DatasetFinal.csv", index_col=0)
df.head(2)

,Course Title,Rating,Level,Schedule,What you will learn,Skill gain,Modules,Instructor,Offered By,Keyword,Course Url,Duration to complete (Approx.),Number of Review
0,Fashion as Design,4.8,Beginner level,Flexible schedule,Not specified,"Art History, Art, History, Creativity","Introduction, Heroes, Silhouettes, Coutures, L...","Anna Burckhardt, Paola Antonelli, Michelle Mil...",The Museum of Modern Art,Arts and Humanities,https://www.coursera.org/learn/fashion-design,20.0,2813
1,Modern American Poetry,4.4,Beginner level,Flexible schedule,Not specified,Not specified,"Orientation, Module 1, Module 2, Module 3, Mod...",Cary Nelson,University of Illinois at Urbana-Champaign,Arts and Humanities,https://www.coursera.org/learn/modern-american...,34.0,100


# 1. PREPROCESSING

In [3]:
# Listados de ciencias y humanidades:
df_science = df[df["Keyword"] != "Arts and Humanities"]
df_humanities = df[df["Keyword"] == "Arts and Humanities"]

row_humanities = list(df_humanities["Course Title"])
row_science = list(df_science["Course Title"][:476])
all_rows = (row_science) + (row_humanities)

print(type(all_rows), len(all_rows))

# Combination of humanities and science labels
y = np.append(np.ones((len(row_science), 1)), np.zeros((len(row_humanities), 1)), axis=0)

<class 'list'> 943


### Train test split: 20% will be in the test set, and 80% in the training set.

In [4]:
# Conjunto de entrenamiento
def get_data_train(title,label):
    # Obtenemos el 80% del conjunto de datos como conjunto de entrenamiento."
    train_x, test_x, train_y, test_y = train_test_split(title, label, test_size=0.20, random_state=42)

    return train_x, test_x, train_y, test_y

train_x, test_x, train_y, test_y = get_data_train(all_rows,y)
print(type(train_x), len(train_x), len(test_x))
print(type(test_y), len(train_y), len(test_y))

<class 'list'> 754 189
<class 'numpy.ndarray'> 754 189


### Word Count - frequency dictionary

In [5]:
# create frequency dictionary
freqs = fun.build_freqs(train_x, train_y)

# check data type
print(f'type(freqs) = {type(freqs)}')

# check length of the dictionary
print(f'len(freqs) = {len(freqs)}')

type(freqs) = <class 'dict'>
len(freqs) = 1359


# 2. Training a Naive Bayes 

In [6]:
logprior, loglikelihood = fun.train_naive_bayes(freqs, train_x, train_y)
print("Logprior = ", logprior)
print("len(loglikelihood) = ", len(loglikelihood))

La cantidad de palabras únicas es: 1202
El número de cursos es:  754
Logprior =  0.031832926573210685
len(loglikelihood) =  1202


In [7]:
loglikelihood.items()

dict_items([('stori', -1.9825769257441233), ('pm', 0.6564804038711354), ('retail', 0.6564804038711354), ('own', 0.36879833141935453), ('拍摄：像剪辑师一样拍摄', -0.7298139572487552), ('unicamp', -0.7298139572487552), ('proyecto', -0.4421318847969743), ('cv', 0.6564804038711354), ('márquez', -1.1352790653569196), ('中级汉语', 0.6564804038711354), ('piano', -2.233891354025029), ('past', -0.7298139572487552), ('corto', -0.7298139572487552), ('confid', -0.036666776688809924), ('mastertrack', -0.7298139572487552), ('distribut', 1.0619455119792998), ('mean', -0.7298139572487552), ('air', -0.7298139572487552), ('busi', 4.257211471208366), ('food', -0.036666776688809924), ('wrote', -0.7298139572487552), ('informacion', -0.7298139572487552), ('storytel', -0.7298139572487552), ('aborigin', -0.7298139572487552), ('musicianship', -1.828426245916865), ('powerpoint', 0.6564804038711354), ('miro', 1.3496275844310808), ('crear', -1.1352790653569196), ('face', -0.7298139572487552), ('iii-i', -0.7298139572487552), ('g

## 3 - Test Naive Bayes - Predict your course

In [8]:
new_course = "Sports Business Analysis"
#new_course = "Children in Primary Education" 
#new_course = "Writing the best essay for university interviews about education"

fun.naive_bayes_predict(new_course, logprior, loglikelihood)

7.343420074451084

In [9]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood, naive_bayes_predict=fun.naive_bayes_predict):
    """
    Input:
        test_x: A list of titles
        test_y: the corresponding labels for the list of titles
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Output:
        accuracy: (# of titles classified correctly)/(total # of titles)
    """
    accuracy = 0  # return this properly
    y_hats = []
    for title in test_x:
        # if the prediction is > 0
        if naive_bayes_predict(title, logprior, loglikelihood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            y_hat_i = 0
        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    error = np.mean(np.absolute(y_hats - test_y))

    # Accuracy is 1 minus the error
    accuracy = 1 - error

    return accuracy

print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

Naive Bayes accuracy = 0.4989


The accuracy is very low. Logistic regression is giving us much better predictions.

In [23]:
# Some error analysis done for you
print('Truth Predicted Title')
for x, y in zip(test_x, test_y):
    y_hat = fun.naive_bayes_predict(x, logprior, loglikelihood)
    if y != (np.sign(y_hat) > 0):
        print('%d\t%0.2f\t%s' % (y, np.sign(y_hat) > 0, ' '.join(
            fun.preprocessing(x)).encode('ascii', 'ignore')))
        print(x)

Truth Predicted Title
1	0.00	b'the scienc success what research know you should know'
The Science of Success: What Researchers Know that You Should Know
0	1.00	b''
游戏产业概论
0	1.00	b'creacin beat patron rtmico'
Creación de beats (patrones rítmicos)
0	1.00	b'learn how learn power mental tool master tough subject'
Learning How to Learn: Powerful mental tools to help you master tough subjects
0	1.00	b' zhuangzipostur awar sentiment'
莊子─姿勢、意識與感情 (Zhuangzi─Posture, Awareness, and Sentiment)
0	1.00	b'la philanthropi comprendr agir'
La Philanthropie : Comprendre et Agir
0	1.00	b'grundlagen grafikdesign'
Grundlagen des Grafikdesigns
0	1.00	b'distribucin digit msica independient'
Distribución digital de la música independiente


/var/folders/np/2h53vflx42jd3w_yxn33bvv40000gn/T/ipykernel_80347/1187023751.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('%d\t%0.2f\t%s' % (y, np.sign(y_hat) > 0, ' '.join(


1	0.00	b'understand music busi what music worth'
Understanding the Music Business: What is Music Worth?
0	1.00	b'habilidad pensamiento solucin problema'
Habilidades de pensamiento y solución de problemas
0	1.00	b'herencia negra aplicacin ritmo afroperuano'
Herencia negra, aplicación de ritmos afroperuanos
0	1.00	b'web design strategi inform architectur'
Web Design: Strategy and Information Architecture
0	1.00	b''
品读道家的智慧
0	1.00	b'becom journalist report new special'
Become a Journalist: Report the News! Specialization
0	1.00	b'build your career profession singer'
Building Your Career as a Professional Singer
0	1.00	b'pensamiento cientfico'
Pensamiento científico
0	1.00	b'teach violin viola creat healthi foundat'
Teaching the Violin and Viola: Creating a Healthy Foundation
0	1.00	b'commun public'
Communicating with the Public
0	1.00	b''
透过摄影看世界


There are quite a few errors that are predicted correctly with logistic regression.